# Perceptron Algorithm from Scratch

This notebook implements the **Perceptron learning algorithm** from scratch using only NumPy — no sklearn for the core logic.

The Perceptron is one of the earliest supervised learning algorithms. It updates weights iteratively based on misclassified points, making it a great foundation for understanding gradient-based learning.

### What this notebook covers
- Manual weight update loop (the core perceptron rule)
- Training on **linearly separable** data — convergence guaranteed
- Training on **linearly inseparable** data — demonstrates the algorithm's fundamental limitation
- Visualising the decision boundary and misclassification curve per epoch
- Comparing the manual implementation against `sklearn`'s `Perceptron`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Perceptron
from sklearn.datasets import make_classification, make_circles

## Core Implementation

The perceptron update rule:
- If predicted **1** but actual **0**: subtract the feature vector from weights
- If predicted **0** but actual **1**: add the feature vector to weights
- Correct predictions: no update

In [ ]:
def train_perceptron(X, y, n_epochs=30, initial_weights=None):
    """
    Train a perceptron using the standard weight update rule.

    Parameters
    ----------
    X : ndarray of shape (n_samples, n_features)
    y : ndarray of shape (n_samples,) with binary labels 0 or 1
    n_epochs : int — number of full passes through the dataset
    initial_weights : ndarray or None — starting weights (zeros if None)

    Returns
    -------
    weights : final weight vector
    errors_per_epoch : list of misclassification counts per epoch
    """
    weights = initial_weights.copy() if initial_weights is not None else np.zeros(X.shape[1])
    errors_per_epoch = []

    for _ in range(n_epochs):
        over_errors = 0    # predicted 1, actual 0
        under_errors = 0   # predicted 0, actual 1

        for i in range(len(X)):
            prediction = np.dot(weights, X[i])

            if prediction > 0 and y[i] == 0:
                weights -= X[i]
                over_errors += 1
            elif prediction <= 0 and y[i] == 1:
                weights += X[i]
                under_errors += 1

        errors_per_epoch.append(over_errors + under_errors)

    return weights, errors_per_epoch


def plot_results(X, y, weights, errors_per_epoch, title_suffix=''):
    """Plot the decision boundary and training error curve side by side."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Decision boundary
    ax1.scatter(X[y == 0, 0], X[y == 0, 1], color='steelblue', label='Class 0', alpha=0.7, edgecolors='k', linewidths=0.4)
    ax1.scatter(X[y == 1, 0], X[y == 1, 1], color='tomato', label='Class 1', alpha=0.7, edgecolors='k', linewidths=0.4)

    if weights[1] != 0:
        x1_vals = np.linspace(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5, 200)
        x2_vals = -(weights[0] / weights[1]) * x1_vals
        ax1.plot(x1_vals, x2_vals, color='green', linewidth=2, label='Decision Boundary')

    ax1.set_title(f'Decision Boundary {title_suffix}', fontsize=13)
    ax1.set_xlabel('Feature 1')
    ax1.set_ylabel('Feature 2')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # Training error
    ax2.plot(range(len(errors_per_epoch)), errors_per_epoch, marker='o', color='steelblue', linewidth=1.5, markersize=4)
    ax2.set_title(f'Misclassifications per Epoch {title_suffix}', fontsize=13)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Misclassifications')
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()
    print(f"Final weights: {weights}")
    print(f"Final misclassifications: {errors_per_epoch[-1]}")

## Part 1: Linearly Separable Data

The **Perceptron Convergence Theorem** guarantees that if the classes are linearly separable, the algorithm will find a perfect boundary in a finite number of steps.

In [ ]:
np.random.seed(42)
X_sep, y_sep = make_classification(
    n_samples=200, n_features=2, n_redundant=0,
    n_clusters_per_class=1, class_sep=2.0, random_state=42
)

weights_sep, errors_sep = train_perceptron(X_sep, y_sep, n_epochs=30)
plot_results(X_sep, y_sep, weights_sep, errors_sep, '(Linearly Separable)')

## Part 2: Linearly Inseparable Data

When classes cannot be separated by a straight line, the perceptron **never converges** — it oscillates indefinitely. This limitation directly motivated the development of multi-layer neural networks.

In [ ]:
X_insep, y_insep = make_circles(n_samples=200, noise=0.1, factor=0.5, random_state=42)

weights_insep, errors_insep = train_perceptron(
    X_insep, y_insep, n_epochs=100, initial_weights=np.array([1.0, -3.0])
)
plot_results(X_insep, y_insep, weights_insep, errors_insep, '(Linearly Inseparable)')

## Part 3: Comparison with sklearn's Perceptron

We verify our manual implementation by comparing its decision boundary against sklearn's `Perceptron`.

In [ ]:
sk_perceptron = Perceptron(max_iter=30, tol=None, random_state=42)
sk_perceptron.fit(X_sep, y_sep)

sk_weights = sk_perceptron.coef_[0]
sk_intercept = sk_perceptron.intercept_[0]

print(f"sklearn accuracy:               {sk_perceptron.score(X_sep, y_sep):.2%}")
print(f"Manual final misclassifications: {errors_sep[-1]}")

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(X_sep[y_sep == 0, 0], X_sep[y_sep == 0, 1], color='steelblue', label='Class 0', alpha=0.7)
ax.scatter(X_sep[y_sep == 1, 0], X_sep[y_sep == 1, 1], color='tomato', label='Class 1', alpha=0.7)

x1_vals = np.linspace(X_sep[:, 0].min(), X_sep[:, 0].max(), 200)

if weights_sep[1] != 0:
    ax.plot(x1_vals, -(weights_sep[0] / weights_sep[1]) * x1_vals,
            color='green', linewidth=2, label='Manual Perceptron')

if sk_weights[1] != 0:
    ax.plot(x1_vals, -(sk_weights[0] * x1_vals + sk_intercept) / sk_weights[1],
            color='orange', linewidth=2, linestyle='--', label='sklearn Perceptron')

ax.set_title('Manual vs sklearn Decision Boundary', fontsize=13)
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

| | Linearly Separable | Linearly Inseparable |
|---|---|---|
| Convergence | ✅ Guaranteed | ❌ Never converges |
| Final errors | 0 | Oscillates |
| Decision boundary | Stable | Keeps shifting |

The perceptron's failure on non-linearly separable data was a key motivation for multi-layer neural networks and non-linear activation functions.